In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import warnings; warnings.simplefilter('ignore')

In [36]:
#list of movies.
movie = pd. read_csv('movies_data_chart.csv')
movie.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-1995,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-1995,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-1995,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-1995,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-1995,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [37]:
#List of rating.
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [38]:
#List of keywords.
keywords = pd.read_csv('keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [39]:
#List of credits.
credits = pd.read_csv('credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [24]:
#weighted rating formula (weighted Rating (WR) = (vv+m.R)+(mv+m.C))
movie['genres'] = movie['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [26]:
movie['year'] = pd.to_datetime(movie['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [34]:
#calculating the no of qualified data
qualified = movie[(movie['vote_count'] >= m) & (movie['vote_count'].notnull()) & (movie['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(130, 6)

In [40]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [41]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [44]:
qualified = qualified.sort_values('wr', ascending=False).head(150)

In [45]:
#Displaying the Top 20 Movies
qualified.head(20)

,title,year,vote_count,vote_average,popularity,genres,wr
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.661577
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.650826
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.643159
256,Star Wars,1977,6778,8,42.149697,"[Adventure, Action, Science Fiction]",7.583881
1225,Back to the Future,1985,6239,8,25.778509,"[Adventure, Comedy, Science Fiction, Family]",7.554760
834,The Godfather,1972,6024,8,41.109264,"[Drama, Crime]",7.541974
1154,The Empire Strikes Back,1980,5998,8,19.470959,"[Adventure, Action, Science Fiction]",7.540378
46,Se7en,1995,5915,8,18.457430,"[Crime, Mystery, Thriller]",7.535207
359,The Lion King,1994,5520,8,21.605761,"[Family, Animation, Drama]",7.508916
586,The Silence of the Lambs,1991,4549,8,4.307222,"[Crime, Drama, Thriller]",7.429601


In [57]:
#reading the list using link
links_small = pd.read_csv('link.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [58]:
movie['id'] = movie['id'].astype('int')

In [59]:
smovie = movie[movie['id'].isin(links_small)]
smovie.shape

(2168, 24)

In [61]:
#content and discription based recommender
smovie['tagline'] = smovie['tagline'].fillna('')
smovie['description'] = smovie['overview'] + smovie['tagline']
smovie['description'] = smovie['description'].fillna('')

In [63]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smovie['description'])
tfidf_matrix.shape

(2168, 77375)

In [64]:
#cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.0082304 , 0.        , ..., 0.        , 0.        ,
       0.00531631])

In [66]:
smovie = smovie.reset_index()
titles = smovie['title']
indices = pd.Series(smovie.index, index=smovie['title'])

In [67]:
#defining a recommendation function that returns the top list of movies
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [69]:
get_recommendations('The Lion King').head(10)

514                       Andre
371                Black Beauty
783           The African Queen
41                  Restoration
1604    Flight of the Navigator
976                     Henry V
602                     Flipper
1747          Shadow of a Doubt
1054               Paris, Texas
1960                 The Fly II
Name: title, dtype: object

In [76]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movie['id'] = movie['id'].astype('int')
movie.shape

(2599, 24)

In [77]:
#merging two datasets
movie = movie.merge(credits, on='id')
movie = movie.merge(keywords, on='id')
smovie = movie[movie['id'].isin(links_small)]
smovie.shape

(2180, 27)

In [78]:
#including cast, crew, keyword in one dataframe
smovie['cast'] = smovie['cast'].apply(literal_eval)
smovie['crew'] = smovie['crew'].apply(literal_eval)
smovie['keywords'] = smovie['keywords'].apply(literal_eval)
smovie['cast_size'] = smovie['cast'].apply(lambda x: len(x))
smovie['crew_size'] = smovie['crew'].apply(lambda x: len(x))

In [81]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [82]:
smovie['director'] = smovie['crew'].apply(get_director)

In [83]:
smovie['cast'] = smovie['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smovie['cast'] = smovie['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smovie['keywords'] = smovie['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [84]:
smovie['cast'] = smovie['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smovie['director'] = smovie['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smovie['director'] = smovie['director'].apply(lambda x: [x,x, x])

In [86]:
s = smovie.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [87]:
s = s.value_counts()
s[:10]

independent film    179
woman director      133
murder              103
based on novel       83
suspense             72
love                 65
violence             64
musical              59
friendship           54
nudity               53
Name: keyword, dtype: int64

In [88]:
#converting word into the rootword
s = s[s > 1]
stemmer = SnowballStemmer('english')
stemmer.stem('Lion')

'lion'

In [89]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [91]:
smovie['keywords'] = smovie['keywords'].apply(filter_keywords)
smovie['keywords'] = smovie['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smovie['keywords'] = smovie['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [105]:
smovie['soup'] = smovie['keywords'] + smovie['cast'] + smovie['director']
smovie['soup'] = smovie['soup'].apply(lambda x: ' '.join(x))

In [106]:
#using countVectorizer to create count_matrix
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smovie['soup'])

In [107]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [108]:
smovie = smovie.reset_index()
titles = smovie['title']
indices = pd.Series(smovie.index, index=smovie['title'])

In [109]:
#listing the suggestions
get_recommendations('Toy Story').head(10)

1875              A Bug's Life
854         That Thing You Do!
192           Boys on the Side
25                     Othello
1022               Stand by Me
1478    Dream for an Insomniac
137                     Casper
320            The Flintstones
1562              Child's Play
321               Forrest Gump
Name: title, dtype: object

In [111]:
get_recommendations('The Lion King').head(10)

7                     Tom and Huck
246               Man of the House
382                    Clean Slate
38        Cry, the Beloved Country
555                 A Family Thing
1264                  Wild America
464          The Road to Wellville
1005                         Glory
1246                   Rough Magic
1861    I'll Be Home for Christmas
Name: title, dtype: object

In [121]:
#defining improved recommendations
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smovie.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [122]:
improved_recommendations('The Lion King')

,title,vote_count,vote_average,wr
1005,Glory,391,7,5.939460
1067,"Paris, Texas",282,7,5.873215
1693,The Addams Family,871,6,5.784387
1268,Conan the Barbarian,663,6,5.763292
361,Addams Family Values,537,6,5.748380
567,James and the Giant Peach,375,6,5.726202
1062,My Life as a Dog,70,7,5.717390
1240,Addicted to Love,112,6,5.680482
1789,Toys,173,5,5.586650
652,The Cable Guy,603,5,5.464550


In [123]:
#Based on the ratings
ratings= pd.read_csv("ratings.csv")
ratings= ratings[['userId', 'movieId', 'rating']]
ratings=ratings.head(1000000)

In [125]:
meta= pd.read_csv("movies_data_chart.csv")
meta= meta[['id', 'original_title', 'original_language']]
meta= meta.rename(columns={'id':'movieId'})
meta = meta[meta['original_language']== 'en'] 
meta

,movieId,original_title,original_language
0,862,Toy Story,en
1,8844,Jumanji,en
2,15602,Grumpier Old Men,en
3,31357,Waiting to Exhale,en
4,11862,Father of the Bride Part II,en
...,...,...,...
2594,10208,Muppets from Space,en
2595,2667,The Blair Witch Project,en
2596,48958,My Life So Far,en
2597,345,Eyes Wide Shut,en


In [133]:
meta.movieId =pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors= 'coerce')
data= pd.merge(ratings, meta, on='movieId', how='inner')
data.head()

,userId,movieId,rating,original_title,original_language
0,1,6,4.0,Judgment Night,en
1,6,6,4.0,Judgment Night,en
2,11,6,5.0,Judgment Night,en
3,18,6,4.0,Judgment Night,en
4,23,6,4.0,Judgment Night,en


In [134]:
user_ratings= data.pivot_table(index='userId', columns='original_title', values='rating')
user_ratings.head()

original_title,10 Things I Hate About You,12 Angry Men,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,3 Ninjas: High Noon at Mega Mountain,54,A Clockwork Orange,A Close Shave,A Few Good Men,...,When Harry Met Sally...,When Night Is Falling,While You Were Sleeping,Wild Things,Wild Wild West,Willy Wonka & the Chocolate Factory,Working Girl,Young Frankenstein,Young and Innocent,eXistenZ
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN


In [135]:
##Let's Remove Movies which have less than 10 users who rated it. and fill remaining NaN with 0
user_ratings = user_ratings.dropna(thresh=10,axis=1).fillna(0)

In [136]:
##Let's Build our Similarity Matrix
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head(50)

original_title,"20,000 Leagues Under the Sea",2001: A Space Odyssey,A Clockwork Orange,A Close Shave,A Nightmare on Elm Street,A Perfect Murder,A Time to Kill,A View to a Kill,Addicted to Love,Alien,...,Touch of Evil,Tron,True Romance,Wag the Dog,We're No Angels,What's Eating Gilbert Grape,While You Were Sleeping,Willy Wonka & the Chocolate Factory,Young Frankenstein,Young and Innocent
original_title,,,,,,,,,,,,,,,,,,,,,
"20,000 Leagues Under the Sea",1.000000,0.123208,0.282372,0.083546,0.230321,0.088179,0.176860,0.079519,0.123308,0.144216,...,0.066417,0.083968,0.099559,0.205867,0.010052,0.155131,0.043387,0.259543,0.104886,0.039611
2001: A Space Odyssey,0.123208,1.000000,0.238790,0.030973,0.187337,0.011873,0.017098,0.156860,0.070285,0.140205,...,-0.013785,-0.038635,0.026118,0.121594,0.005972,0.045331,0.018116,0.281035,0.001994,-0.027992
A Clockwork Orange,0.282372,0.238790,1.000000,0.028049,0.311745,0.033723,0.160284,0.090787,0.144418,0.048088,...,-0.008539,0.035442,0.019756,0.224079,0.007416,0.074517,0.029583,0.322722,0.042789,0.030742
A Close Shave,0.083546,0.030973,0.028049,1.000000,0.107089,0.036679,-0.027464,-0.029540,-0.035782,0.121109,...,0.022293,0.032331,0.119912,0.117350,-0.040819,0.038773,0.024502,0.044158,0.063886,-0.033009
A Nightmare on Elm Street,0.230321,0.187337,0.311745,0.107089,1.000000,0.021445,0.159925,0.069747,0.206707,0.179645,...,0.077552,0.075677,0.091797,0.382798,0.050634,0.131205,0.102264,0.215879,0.117490,0.153719
A Perfect Murder,0.088179,0.011873,0.033723,0.036679,0.021445,1.000000,0.058915,0.096733,0.125803,0.266385,...,0.044155,0.105837,0.073249,0.100218,0.043304,0.242842,0.311415,0.025926,0.114730,0.120593
A Time to Kill,0.176860,0.017098,0.160284,-0.027464,0.159925,0.058915,1.000000,0.032312,0.237322,0.130191,...,0.192595,0.143344,0.091424,0.122274,0.089107,0.146191,0.114053,0.070750,0.121112,0.293904
A View to a Kill,0.079519,0.156860,0.090787,-0.029540,0.069747,0.096733,0.032312,1.000000,0.076621,0.060169,...,0.016171,0.020699,0.068357,0.007345,-0.005829,0.078988,0.097531,0.077305,-0.009960,-0.048598
Addicted to Love,0.123308,0.070285,0.144418,-0.035782,0.206707,0.125803,0.237322,0.076621,1.000000,0.055169,...,0.162702,0.096911,0.063848,0.124493,0.052460,0.255885,0.085244,0.074667,0.117968,0.193145


In [137]:
##creating the similar movie recommendations function
def get_similar_movies(movie_name,user_rating):
    similar_score = item_similarity_df[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score

In [141]:
#giving ratings for the movies
action_lover = [("Blackmail",5)
                ,("Color of Night",3),("A Clockwork Orange",4),
               ("American Pie",1)]

similar_movies = pd.DataFrame()

for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating),ignore_index=True)
#listing the recommendations    
similar_movies.head()
similar_movies.sum().sort_values(ascending=False)

Blackmail              2.583733
A Clockwork Orange     1.869763
Romeo + Juliet         1.143190
Night on Earth         1.064737
Rain Man               1.062154
                         ...   
Say Anything...       -0.091950
One Tough Cop         -0.101710
Touch of Evil         -0.164870
Tomorrow Never Dies   -0.334260
American Pie          -0.602140
Length: 170, dtype: float64